In [1]:
import warnings
import requests
import folium
from pandas.io.json import json_normalize

# 스타벅스 사이트에서 위치정보를 수집
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=5XLKAT2F0K'
request = requests.post(targetSite, data={
    'ins_lat'   : 37.5108295,           # 위도
    'ins_lng'   : 127.02928809999999,   # 경도
    'p_sido_cd' : '01',                 # 시,도 분류 코드
    'p_gugun_cd': '',                   # 시,군,구 분류 코드
    'in_biz_cd' : '' ,                  # 알 수 없는 정보(불필요)
    'iend'      : 2000,                 # 불러올 최대 매장 수
    'set_date'  : ''                    # 알 수 없는 정보(불필요)
})

star_json = request.json()
star_df = json_normalize(star_json, 'list')


<ipython-input-1-e64239d48825>:19: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  star_df = json_normalize(star_json, 'list')


In [2]:
star_df = json_normalize(star_json, 'list')
print(type(star_df))
print(len(star_df))
star_df

<class 'pandas.core.frame.DataFrame'>
612


<ipython-input-2-daccd2f8c8c0>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  star_df = json_normalize(star_json, 'list')


,seq,sido_cd,sido_nm,gugun_cd,gugun_nm,code_order,view_yn,store_num,sido,gugun,...,p90,p01,t05,t30,t36,t27,t29,t43,t48,z9999
0,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
1,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
2,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
3,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
4,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
608,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
609,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
610,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0


In [3]:
star_df_map = star_df[['s_name', 'sido_code', 'sido_name', 'gugun_code', 'gugun_name'
                       , 'doro_address', 'lat', 'lot']]

# astype() 메소드로 위도, 경도의 데이터 타입—>object에서 float.
star_df_map['lat'] = star_df_map['lat'].astype(float)
star_df_map['lot'] = star_df_map['lot'].astype(float)

<ipython-input-3-10028af16f16>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  star_df_map['lat'] = star_df_map['lat'].astype(float)
<ipython-input-3-10028af16f16>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  star_df_map['lot'] = star_df_map['lot'].astype(float)


In [4]:
star_map = folium.Map(
    location=[star_df_map['lat'].mean(),star_df_map['lot'].mean()],
    zoom_start=12)
for (index, data) in star_df_map.iterrows():
    string = '{}점 - {}'.format(data['s_name'], data['doro_address'])
    popup = folium.Popup(string, max_width=600)
    folium.Marker(
        location=[data['lat'], data['lot']],
        popup=popup
    ).add_to(star_map)
star_map

In [5]:
import pandas as pd
import folium

In [17]:
df = pd.read_csv('서울특별시 자치구별 코로나19 백신접종현황.csv', encoding='cp949')
df.head()

,자치구,2-6월 1차접종자,2-6월 2차접종자,7월말(누적) 1차접종자,7월말(누적) 2차접종자,8월말(누적) 1차접종자,8월말(누적) 2차접종자,9월말(누적) 1차접종자,9월말(누적) 2차접종자,10월말(누적) 1차접종자,10월말(누적) 2차접종자,10월말(누적) 추가접종
0,종로구,64067,28168,79113,40119,114242,67930,113892,74683,118945,112459,198
1,중구,55134,18966,71425,27964,83716,48807,99187,63028,104060,97949,724
2,용산구,65070,23205,82264,32113,122796,67814,173186,114336,180716,170709,184
3,성동구,85252,29483,107896,38956,162919,87625,224883,144948,233290,221269,1710
4,광진구,96935,32176,126826,42969,193076,102210,273100,169311,285027,268408,276


/content/drive/MyDrive/Colab Notebooks/SKT_FLY_AI_4th/Week_1/Day_2
